In [25]:
import sys
import os
from dotenv import load_dotenv

# --- 1. DYNAMIC PATH SETUP ---
# Get the current working directory (where this notebook is running)
current_dir = os.getcwd()

# Go up one level to find the Project Root (assuming notebook is in /notebooks)
# If your notebook is in the root, remove the next line and use project_root = current_dir
project_root = os.path.dirname(current_dir)

# Define the 'src' folder path relative to the root
src_path = os.path.join(project_root, 'src')

# Add 'src' to system path so imports work
if src_path not in sys.path:
    sys.path.append(src_path)

# --- 2. LOAD ENVIRONMENT VARIABLES ---
# Look for .env file in the project root or src
# (This is safer than hardcoding E:\...)
env_path = os.path.join(project_root, '.env')
if not os.path.exists(env_path):
    # Fallback: check src folder if not in root
    env_path = os.path.join(src_path, '.env')

load_dotenv(env_path)

# --- 3. IMPORTS ---
try:
    from pdf_parser import extract_text_from_pdf
    from agent import RecruitmentAgent
    from utils import generate_email
    print("✅ Libraries imported successfully.")
except ImportError as e:
    print(f"❌ Import Error: {e}")
    print(f"   Debug: Python is looking in: {sys.path}")

# --- 4. DATA PATHS ---
# Define paths relative to the project root
resume_folder = os.path.join(project_root, "data", "resumes")
jd_path = os.path.join(project_root, "data", "job_description.txt")

# Initialize Agent
try:
    agent = RecruitmentAgent()
    print("✅ Agent initialized.")
except Exception as e:
    print(f"❌ Agent Error: {e} (Check your API Key)")

# Read Job Description
try:
    with open(jd_path, "r") as f:
        jd_text = f.read()
    print("✅ Job Description loaded successfully.")
except FileNotFoundError:
    # Handle the case where the file might be named .txt.txt by accident
    print(f"❌ Error: Could not find JD at {jd_path}")
    print("   Tip: Check if your file is actually named 'job_description.txt.txt'")


# --- 5. THE LOOP ---
print("\n" + "="*60)
print("🚀 STARTING RECRUITMENT PROCESS")
print("="*60 + "\n")

# Get list of resumes
try:
    resumes = [f for f in os.listdir(resume_folder) if f.endswith('.pdf')]
except FileNotFoundError:
    print(f"❌ Error: Resume folder not found at {resume_folder}")
    resumes = []

for filename in resumes:
    print("-" * 60)
    print(f"📄 PROCESSING CANDIDATE: {filename}")
    print("-" * 60)
    
    # A. Extract Text
    resume_path = os.path.join(resume_folder, filename)
    resume_text = extract_text_from_pdf(resume_path)
    
    # B. AI Analysis
    print("🤖 Agent is thinking...")
    analysis = agent.analyze_candidate(resume_text, jd_text)
    
    # Display the AI's opinion
    print("\n--- AI ANALYSIS REPORT ---")
    print(analysis)
    print("--------------------------\n")
    
    # C. HUMAN IN THE LOOP (Verification)
    # Clean up filename to get a human-readable name
    candidate_name = filename.replace(".pdf", "").replace("_", " ")
    
    print(f"👮 HUMAN VERIFICATION REQUIRED FOR: {candidate_name}")
    user_decision = input("Do you agree with the AI's decision? (yes/no): ").lower()
    
    final_decision = ""
    reasoning_summary = ""
    
    if user_decision == 'yes':
        print("✅ Decision Verified.")
        # Logic to parse the AI's unstructured text decision
        if "Interview" in analysis or "Select" in analysis:
            final_decision = "Interview"
            reasoning_summary = "Your skills strongly align with our JD requirements."
        else:
            final_decision = "Reject"
            reasoning_summary = "Skills do not match the current requirements."
            
    else:
        print("⚠️  Decision Overridden by Human.")
        final_decision = input("Enter final decision (Interview/Reject): ")
        reasoning_summary = input("Enter brief feedback for the candidate: ")

    # D. Communication Agent
    print("\n📧 GENERATING EMAIL DRAFT...")
    email_draft = generate_email(candidate_name, final_decision, reasoning_summary)
    
    print(email_draft)
    print("\n" + "="*60 + "\n")

✅ Success: Paths found and Agent loaded!
✅ Agent and Data Loaded Successfully.
------------------------------------------------------------
📄 PROCESSING CANDIDATE: Bad Match.pdf
------------------------------------------------------------
🤖 Agent is thinking...

--- AI ANALYSIS REPORT ---
Here's the evaluation of Casey Creative's resume against the Job Description:

1.  **MATCH SCORE:** 5%
2.  **DECISION:** Reject
3.  **KEY STRENGTHS:**
    *   Team Leadership
    *   Public Speaking
    *   "Out of the Box" Thinking (as a cultural fit element)
4.  **MISSING CRITICAL SKILLS:**
    *   Master's degree in a quantitative field (Math, Physics, Engineering, Statistics, Economics, or Finance)
    *   0-2 years of experience in a quantitative or modeling role
    *   Deep understanding of statistical/econometric models (linear, logistics, time series)
    *   Proficiency in Python, R, or equivalent programming languages
    *   Knowledge in financial markets or prior experience in model devel

Do you agree with the AI's decision? (yes/no):  yes


✅ Decision Verified.

📧 GENERATING EMAIL DRAFT...

        Date: November 30, 2025
        To: Bad Match
        Subject: Update on your application - Bad Match
        
        Dear Bad Match,
        
        Thank you for your interest in joining our team. 
        
        We have reviewed your application, and unfortunately, we have decided to proceed with other candidates who more closely match our specific requirements for this role at this time.
        
        Specific Feedback: Mismatched technical stack.
        
        We wish you the best in your job search.
        
        Sincerely,
        Talent Acquisition Team
        


------------------------------------------------------------
📄 PROCESSING CANDIDATE: Okay Match.pdf
------------------------------------------------------------
🤖 Agent is thinking...

--- AI ANALYSIS REPORT ---
Here is the evaluation of the candidate:

1.  **MATCH SCORE:** 88%
2.  **DECISION:** Select for Interview
3.  **KEY STRENGTHS:**
    *   

Do you agree with the AI's decision? (yes/no):  yes


✅ Decision Verified.

📧 GENERATING EMAIL DRAFT...

        Date: November 30, 2025
        To: Okay Match
        Subject: Invitation to Interview - Okay Match
        
        Dear Okay Match,
        
        We are pleased to inform you that your application has been selected for the next round of interviews! 
        
        Our AI review team was particularly impressed by:
        Your skills strongly align with our JD requirements.
        
        Our scheduling coordinator will be in touch shortly to arrange a time.
        
        Best regards,
        Talent Acquisition Team
        


------------------------------------------------------------
📄 PROCESSING CANDIDATE: Perfect Match.pdf
------------------------------------------------------------
🤖 Agent is thinking...

--- AI ANALYSIS REPORT ---
Here is the evaluation of the candidate:

1.  **MATCH SCORE:** 95%
2.  **DECISION:** Select for Interview
3.  **KEY STRENGTHS:**
    *   Master's degree in Statistics combined with

Do you agree with the AI's decision? (yes/no):  no


⚠️  Decision Overridden by Human.


Enter final decision (Interview/Reject):  role already filled 
Enter brief feedback for the candidate:  no



📧 GENERATING EMAIL DRAFT...

        Date: November 30, 2025
        To: Perfect Match
        Subject: Update on your application - Perfect Match
        
        Dear Perfect Match,
        
        Thank you for your interest in joining our team. 
        
        We have reviewed your application, and unfortunately, we have decided to proceed with other candidates who more closely match our specific requirements for this role at this time.
        
        Specific Feedback: no
        
        We wish you the best in your job search.
        
        Sincerely,
        Talent Acquisition Team
        


